In [1]:
import findspark
import matplotlib.pyplot as plt
findspark.init()
findspark.find()
import pyspark
findspark.find()

'D:\\spark-2.4.6-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf
import numpy as np
import pandas as pd
from pyspark.sql.functions import mean , stddev, col, to_date, date_format
import pyspark.sql.functions as F
from time import strptime
import datetime
from ast import literal_eval


from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark=SparkSession(sc)

In [3]:
tags= sc.textFile('E:/DDA/Exercise_8/tags.dat').map(lambda line: line.split("::")).collect()

In [4]:
rdd_a=sc.parallelize(tags).toDF()
rdd_a = rdd_a.withColumnRenamed("_1", "UserID").withColumnRenamed("_2", "MovieID").withColumnRenamed("_3", "Rating").withColumnRenamed("_4", "Timestamp")

In [5]:
timestamp=[(literal_eval(i.UserID),literal_eval(i.Timestamp)) for i in rdd_a.collect()]

In [6]:
def total_time(tags):
    user_duration = []
    user_duration.append((tags[0][0],0))
    for i in range(1,99):
        if tags[i][0] == tags[i-1][0]:
            duration = tags[i][1] - tags[i+1][1]
            duration = abs(duration)
            duration = duration / 60
            user_duration.append((tags[i][0],duration))
        elif not tags[i][0] == tags[i-1][0]:
            user_duration.append((tags[i][0],0))
    return user_duration

def calculate_user_session(tag_updated_duration):
    x = 1
    session_list = []
    session_list.append((tag_updated_duration[0][0],1))
    for i in range(1,98):
        if tag_updated_duration[i][0] == tag_updated_duration[i+1][0] and tag_updated_duration[i][1] <30:
            x = 1
        elif tag_updated_duration[i][0] == tag_updated_duration[i+1][0] and tag_updated_duration[i][1] >=30:
            x = x + 1
        elif not tag_updated_duration[i][0] == tag_updated_duration[i+1][0]:
            x = 1
        session_list.append((tag_updated_duration[i][0],x))
    return  session_list    

tag_updated_duration = total_time(timestamp[:100])
tags_session = calculate_user_session(tag_updated_duration)
tags_session

[(15, 1),
 (20, 1),
 (20, 1),
 (20, 1),
 (20, 1),
 (20, 1),
 (20, 1),
 (20, 1),
 (20, 1),
 (20, 1),
 (20, 1),
 (20, 1),
 (20, 1),
 (21, 1),
 (21, 1),
 (25, 1),
 (25, 1),
 (31, 1),
 (31, 1),
 (31, 1),
 (31, 1),
 (31, 1),
 (32, 1),
 (39, 1),
 (39, 1),
 (39, 1),
 (39, 1),
 (39, 1),
 (48, 1),
 (48, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (49, 1),
 (75, 1),
 (78, 1),
 (109, 1),
 (109, 1),
 (109, 2),
 (109, 3),
 (109, 4),
 (109, 1),
 (109, 2),
 (109, 1),
 (109, 1),
 (109, 1),
 (109, 2),
 (109, 3),
 (109, 4),
 (109, 1),
 (109, 1),
 (109, 2),
 (109, 1),
 (109, 2),
 (109, 3),
 (109, 4),
 (109, 5),
 (109, 1),
 (109, 2),
 (109, 3),
 (109, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (127, 1),
 (

In [7]:
def calculate_frequency(tag_session):
    dic={}
    for i in range(len(tag_session)):
        if (tag_session[i][0],tag_session[i][1]) in dic:
            dic[(tag_session[i][0],tag_session[i][1])] += 1
        else:
            dic[(tag_session[i][0],tag_session[i][1])] =1
    return dic

frequency_list=calculate_frequency(tags_session)
print(frequency_list)

{(15, 1): 1, (20, 1): 12, (21, 1): 2, (25, 1): 2, (31, 1): 5, (32, 1): 1, (39, 1): 5, (48, 1): 2, (49, 1): 15, (75, 1): 1, (78, 1): 1, (109, 1): 11, (109, 2): 6, (109, 3): 4, (109, 4): 3, (109, 5): 1, (127, 1): 26}


In [10]:
def each_and_across_user(frequency_list):
    sd = {}
    updated = {}
    dic_updated = {}
    complete_data = []
    for key1 in frequency_list:
        if(key1[0] not in dic_updated):
            dic_updated[key1[0]] = []
        dic_updated[key1[0]].append(key1[1])
        complete_data.append(key1[1])    
    for key2 in dic_updated:
        updated[key2] = np.mean(dic_updated[key2])        
        sd[key2] = np.std(dic_updated[key2])
    across_mean = np.mean(complete_data)
    acrosss_sd = np.std(complete_data)
    
    print("Each User Mean: " ,updated)
    print("Each User Sd: " ,sd)
    print("Each Across User Mean: " ,across_mean)
    print("Each Across User Sd: ",acrosss_sd)
    print("Each Final Data: " ,complete_data)



In [11]:
each_and_across_user(frequency_list)

Each User Mean:  {15: 1.0, 20: 1.0, 21: 1.0, 25: 1.0, 31: 1.0, 32: 1.0, 39: 1.0, 48: 1.0, 49: 1.0, 75: 1.0, 78: 1.0, 109: 3.0, 127: 1.0}
Each User Sd:  {15: 0.0, 20: 0.0, 21: 0.0, 25: 0.0, 31: 0.0, 32: 0.0, 39: 0.0, 48: 0.0, 49: 0.0, 75: 0.0, 78: 0.0, 109: 1.4142135623730951, 127: 0.0}
Each Across User Mean:  1.588235294117647
Each Across User Sd:  1.1910856900774465
Each Final Data:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5, 1]
